In [7]:
from context import omphalos

# Import Omphalos modules.
from omphalos import generate_inputs as gi
from omphalos import file_methods as fm
from omphalos import omphalos_plotter as op
from omphalos import attributes as attr
from omphalos import labels as lbls
from omphalos import spatial_constructor

### Import and prep training/test data.
wandb.init(project='xgboost_rifle_sweep')
train_set = fm.unpickle('/Users/angus/Omphalos/fitting/data/old_rifle.pkl')

attributes_df = attr.boundary_condition(train_set, boundary='x_begin')
labels_df = lbls.secondary_precip(train_set)

x = attributes_df.loc[:, ['NH4+', 'SO4--','Ca++', 'Acetate', 'CO2(aq)']]
y = labels_df.sum(level=0)['FeS(am)'] + labels_df.sum(level=0)['FeS34(am)']

x = x.to_numpy()
y = y.to_numpy().reshape(-1,1)
y = y * 1e4

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=69)

In [16]:
train(train_set)

eval-mphe,2.50426
train-mphe,2.46463
_runtime,17
_timestamp,1613057071
_step,1000
rmse,5.48071


eval-mphe,█▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train-mphe,█▇▆▆▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████
_timestamp,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
rmse,▁


/usr/local/lib/python3.8/site-packages/xgboost/training.py:20: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


In [17]:
def train():
    import numpy as np
    import wandb
    
    ### Set random vars.
    np.random.seed(0)
    
    config_defaults = {'eta': 0.0001,
                        'l1': 0,
                        'l2': 1, 
                        'max_depth': 6
                        }

    wandb.init(config=config_defaults, magic=True)

    ### Initialise booster model.
    import xgboost as xgb
    
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_test, label=y_test)

    evallist = [(dtest, 'eval'), (dtrain, 'train')]
    param = {'max_depth': wandb.config.max_depth, 'eta': wandb.config.eta, 'objective': 'reg:pseudohubererror', 'alpha': wandb.config.l1, 'lambda': wandb.config.l2}

    results = {}

    num_round = 1000
    bst = xgb.train(param, dtrain, num_round, evallist, evals_result=results,early_stopping_rounds=500, verbose_eval=False, callbacks=[wandb.xgboost.wandb_callback()])
    
    
    ypred = bst.predict(dtest)

    from sklearn.metrics import mean_squared_error

    rmse = np.sqrt(mean_squared_error(y_test, ypred))
    wandb.log({"rmse": rmse})

In [18]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'rmse',
        'goal': 'minimize'
    },
    'parameters': {
        'eta': {
            'min': 0.001,
            'max': 0.1
        },
        'l1': {
            'min': 0,
            'max': 1
        },
        'l2': {
            'min': 0,
            'max': 10
        },
        'max_depth': {
            'min': 4,
            'max': 10
        }
    }
}

In [20]:
import wandb
sweep_id = wandb.sweep(sweep_config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: zfx7nh2m
Sweep URL: https://wandb.ai/a-fotherby/uncategorized/sweeps/zfx7nh2m


In [21]:
wandb.agent(sweep_id, function=train())

eval-mphe,2.50426
train-mphe,2.46463
_runtime,15
_timestamp,1613057713
_step,1000
rmse,5.48071


eval-mphe,█▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train-mphe,█▇▆▆▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██████
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
rmse,▁


/usr/local/lib/python3.8/site-packages/xgboost/training.py:20: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: Agent Starting Run: jw47v6yd with config:
	eta: 0.02255027496014549
	l1: 0
	l2: 2
	max_depth: 6
wandb: Agent Starting Run: 0yrenovm with config:
	eta: 0.09170662186830833
	l1: 1
	l2: 5
	max_depth: 6
wandb: Agent Starting Run: qakwpjzg with config:
	eta: 0.08475231565506205
	l1: 0
	l2: 5
	max_depth: 8
wandb: Agent Starting Run: tkrlo2y6 with config:
	eta: 0.03771697029066189
	l1: 0
	l2: 1
	max_depth: 5
wandb: Agent Starting Run: irkq9yq4 with config:
	eta: 0.04574268951828001
	l1: 0
	l2: 4
	max_depth: 8


2021-02-11 15:37:22,567 - wandb.wandb_agent - ERROR - Detected 5 failed runs in a row, shutting down.
